In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

# Preparar o dataset
iris = datasets.load_iris()
X = iris.data; y = iris.target

X = X[y != 1] ; y = y[y != 1] # versicolor
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
y[y == 0] = -1  # SVM espera rótulos em {-1, +1}

X = torch.tensor(X, dtype=torch.float32) # Tensor é um tipo especial que suporta muitas dimensões

(150, 4)
(150,)


In [63]:
print(X.shape)
print(y.shape)

torch.Size([100, 4])
torch.Size([100, 1])


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
# Definir parâmetros treináveis da Support Vector Machine: w e b
n_features = X_train.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# === Hiperparâmetros ===
learning_rate = 0.01
epochs = 300
optimizer = optim.Adam([w, b], lr=learning_rate)

In [55]:
for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = X_train @ w + b  # Modelo SVM (um hiperplano que depende de w e b)

    # Hinge loss: max(0, 1 - y_i * (w^T x_i + b))
    perda_de_classificacao = torch.clamp(1 - y_train.view(-1, 1) * y_pred, min=0).mean()

    # Termo de regularização
    perda_de_distancia_entre_classes = 0.5 * torch.sum(w ** 2) # 2/||w|| é a distância que queremos que seja a maior possível

    # Função objetivo tradicional: minimizar reg + C * hinge
    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

Epoch 100/300, Loss=1.5666
Epoch 200/300, Loss=0.5636
Epoch 300/300, Loss=0.1982
